In [1]:
from codeflare_sdk import Cluster, ClusterConfiguration, TokenAuthentication

In [2]:
import os
import sys

In [3]:
# Authenticate the CodeFlare SDK
# On OpenShift, you can retrieve the token by running `oc whoami -t`,
# and the server with `oc cluster-info`.


auth = TokenAuthentication(
    token = os.getenv("AUTH_TOKEN"),
    server = os.getenv("API_SERVER"),
    skip_tls=True
)
auth.login()

Insecure request warnings have been disabled
Action not permitted, have you put in correct/up-to-date auth credentials?


In [11]:
# Configure the Ray cluster
cluster = Cluster(ClusterConfiguration(
    name='ray',
    namespace='distributed-training-demo',
    num_workers=5,
    min_cpus=8,
    max_cpus=8,
    head_cpus=16,
    min_memory=48,
    max_memory=48,
    head_memory=48,
    head_gpus=1,
    num_gpus=1,
    image="quay.io/rhoai/ray:2.23.0-py39-cu121-2024.1",
    local_queue="local-queue-ray",
))

Yaml resources loaded for ray


In [12]:
# Create the Ray cluster
cluster.up()

In [13]:
cluster.wait_ready()

Waiting for requested resources to be set up...
Requested cluster is up and running!
Dashboard is ready!


In [15]:
cluster.details()

                         🚀 CodeFlare Cluster Details 🚀                        
                                                                                
 ╭────────────────────────────────────────────────────────────────────────────╮ 
 │   Name                                                                     │ 
 │   ray                                                          Active ✅   │ 
 │                                                                            │ 
 │   URI: ray://ray-head-svc.distributed-training-demo.svc:10001              │ 
 │                                                                            │ 
 │   ]8;id=300454;https://ray-dashboard-ray-distributed-training-demo.apps.cluster-mpcjw.mpcjw.sandbox2589.opentlc.com\Dashboard🔗]8;;\                                                              │ 
 │                                                                            │ 
 │                       Cluster Resources                                    │ 
 │   ╭── Workers ──╮  ╭───────── Worker specs(each) ─────────╮                │ 
 │   │  # Workers  │  │  Memory      CPU         GPU         │                │ 
 │   │             │  │                                      │                │ 
 │   │  5          │  │  48G~48G     8           1           │                │ 
 │   │             │  │                                      │                │ 
 │   ╰─────────────╯  ╰──────────────────────────────────────╯                │ 
 ╰────────────────────────────────────────────────────────────────────────────╯

RayCluster(name='ray', status=<RayClusterStatus.READY: 'ready'>, head_cpus=16, head_mem='48G', head_gpu=1, workers=5, worker_mem_min='48G', worker_mem_max='48G', worker_cpu=8, worker_gpu=1, namespace='distributed-training-demo', dashboard='https://ray-dashboard-ray-distributed-training-demo.apps.cluster-mpcjw.mpcjw.sandbox2589.opentlc.com')

In [16]:
# Initialize the Job Submission Client
client = cluster.job_client

In [17]:
# Create the training and evaluation datasets.
# This can be run only once.
!{sys.executable} -m pip install datasets
import create_dataset
create_dataset.main()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 10.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.8/193.8 kB 263.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.5/417.5 kB 261.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 249.5 MB/s eta 0:00:00

[notice] A new release of pip available: 22.2.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [18]:
# The S3 bucket where to store checkpoint.
# It can be set manually, otherwise it's retrieved from configured the data connection.
s3_bucket = ""
if not s3_bucket:
    s3_bucket = os.environ.get('AWS_S3_BUCKET')
assert s3_bucket, "An S3 bucket must be provided to store checkpoints"

In [20]:
submission_id = client.submit_job(
    entrypoint="python ray_finetune_llm_deepspeed.py "
               "--model-name=meta-llama/Llama-2-7b-chat-hf "
               "--lora "
               "--num-devices=2 "
               "--num-epochs=3 "
               "--ds-config=./deepspeed_configs/zero_3_llama_2_7b.json "
               f"--storage-path={s3_bucket}/ray_finetune_llm_deepspeed/ "
               "--batch-size-per-device=16 "
               "--eval-batch-size-per-device=32 ",
    runtime_env={
        "env_vars": {
            "AWS_ACCESS_KEY_ID": os.environ.get('AWS_ACCESS_KEY_ID'),
            "AWS_SECRET_ACCESS_KEY": os.environ.get('AWS_SECRET_ACCESS_KEY'),
            "AWS_DEFAULT_REGION": 'us-east-1'
        },
        "pip": "requirements.txt",
        "working_dir": "./",
        "excludes": ["/docs/", "*.ipynb", "*.md", "/tmp/"]
    },
)
print(submission_id)

2024-08-15 09:16:19,850	INFO dashboard_sdk.py:385 -- Package gcs://_ray_pkg_67ae4200f052d56d.zip already exists, skipping upload.


raysubmit_wrZS5wkqmBxX55yg


In [14]:
client.stop_job(submission_id)

True

In [21]:
cluster.down()